In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from pandas.io.json import json_normalize
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import sys
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot as plt
!{sys.executable} -m pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot as plt

In [2]:
with open("df_application_v2.pkl", 'rb') as picklefile: 
    df_application_v2 = pickle.load(picklefile)

In [3]:
with open("credit_card_balance.pkl", 'rb') as picklefile: 
    credit_card_balance_df = pickle.load(picklefile)

In [4]:
credit_card_balance_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [5]:
credit_card_balance_df.loc[credit_card_balance_df["SK_ID_PREV"] == 1891521]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
21235,1891521,126868,-16,453606.390,450000,0.0,21096.000,0.0,21096.000,22429.305,...,453606.390,453606.390,0.0,2,0.0,2.0,86.0,Active,0,0
26770,1891521,126868,-61,169032.645,180000,0.0,8939.250,0.0,8939.250,9000.000,...,169032.645,169032.645,0.0,1,0.0,1.0,41.0,Active,0,0
41137,1891521,126868,-54,73353.420,180000,22500.0,80414.955,0.0,57914.955,1779.975,...,73353.420,73353.420,1.0,5,0.0,4.0,48.0,Active,0,0
56769,1891521,126868,-5,459959.130,450000,0.0,17469.000,0.0,17469.000,22899.645,...,459959.130,459959.130,0.0,1,0.0,1.0,97.0,Active,0,0
180687,1891521,126868,-55,1779.975,180000,0.0,6799.950,0.0,6799.950,9000.000,...,0.000,0.000,0.0,1,0.0,1.0,47.0,Active,0,0
186309,1891521,126868,-65,171704.925,180000,6750.0,15927.570,0.0,9177.570,9000.000,...,171704.925,171704.925,2.0,4,0.0,2.0,37.0,Active,0,0
190537,1891521,126868,-76,174737.430,180000,13500.0,13500.000,0.0,0.000,9000.000,...,174737.430,174737.430,1.0,1,0.0,0.0,26.0,Active,1,1
279889,1891521,126868,-57,175850.910,180000,4500.0,4500.000,0.0,0.000,9000.000,...,175850.910,175850.910,1.0,1,0.0,0.0,45.0,Active,0,0
283473,1891521,126868,-29,149222.250,180000,0.0,26326.530,0.0,26326.530,6648.075,...,149222.250,149222.250,0.0,5,0.0,5.0,73.0,Active,0,0
